In [1]:
import os
import gmsh
import pygmsh
import meshio
import numpy as np

import matplotlib.pyplot as plt
from scipy import interpolate

## Guitar Mesh Setup

In [2]:
# Guitar Parameters
length = 0.48;
lbWidth = 0.365;
ubWidth = 0.28;
indentWidth = 0.23;
lbLoc = 0.13;
indentLoc = 0.31;
ubLoc = 0.41;
topThickness = 0.003;
backThickness = 0.003;
depth = 0.1;
sideThickness = 0.003;
soundholeDiam = 0.08;
soundholeHeight = 0.3;
neckBlockWidth = 0.08;
endBlockWidth = 0.1;

In [3]:
# Outline of points around the guitar top
x = np.array([0, -endBlockWidth, -lbWidth/2, -indentWidth/2, -ubWidth/2, -neckBlockWidth, 0, neckBlockWidth, ubWidth/2, indentWidth/2, lbWidth/2, endBlockWidth])
y = np.array([0, 0, lbLoc, indentLoc, ubLoc, length, length, length, ubLoc, indentLoc, lbLoc, 0])

# Outline of inner points for the sides
xSide = np.array([0, -endBlockWidth, -lbWidth/2+sideThickness, -indentWidth/2+sideThickness, -ubWidth/2+sideThickness, -neckBlockWidth, 0, neckBlockWidth, ubWidth/2-sideThickness, indentWidth/2-sideThickness, lbWidth/2-sideThickness, endBlockWidth])
ySide = np.array([sideThickness, sideThickness, lbLoc, indentLoc, ubLoc, length-sideThickness, length-sideThickness, length-sideThickness, ubLoc, indentLoc, lbLoc, sideThickness])

# append the starting x,y coordinates
x = np.r_[x, x[0]]
y = np.r_[y, y[0]]

xSide = np.r_[xSide, xSide[0]]
ySide = np.r_[ySide, ySide[0]]

In [4]:
# fit splines to x=f(u) and y=g(u), treating both as periodic. also note that s=0
# is needed in order to force the spline fit to pass through all the input points.
tck, u = interpolate.splprep([x, y],task = 0, k = 5, s=0.0008, per=1)
tckSide, uSide = interpolate.splprep([xSide, ySide],task = 0, k = 5, s=0.0008, per=1)

# evaluate the spline fits for 40 evenly spaced distance values
N = 40;
xi, yi = interpolate.splev(np.linspace(0, 1, N), tck)
xiSide, yiSide = interpolate.splev(np.linspace(0, 1, N), tckSide)

In [5]:
# Makes lists of the points for geometry creation
pointList = []
pointListSideInner = []
pointListSideOuter = []

for i in range(1,N):
    pointList.append([xi[i],yi[i],0.0]) 
    pointListSideInner.append([xiSide[i],yiSide[i],0])
    pointListSideOuter.append([xi[i],yi[i],0])

## Guitar Mesh Construction

In [10]:
MeshGeometryFolder = os.path.join("MeshGeometry")

In [13]:
# Create the top mesh
with pygmsh.occ.Geometry() as geom:

    geom.characteristic_length_max = 0.02

    top2D = geom.add_polygon(pointList)
    
    soundhole = geom.add_disk([0.0, soundholeHeight], soundholeDiam/2)
    
    topHole2D = geom.boolean_difference(top2D, soundhole)
    
    top3D = geom.extrude(top2D, [0.0, 0.0, topThickness], num_layers=2)
    
    top3D = [top3D[0], top3D[1]] +  top3D[2]
        
    top_mesh = geom.generate_mesh()

In [7]:
# Create the side mesh
with pygmsh.occ.Geometry() as geom:

    
    geom.characteristic_length_max = 0.02

    sideOuter2D = geom.add_polygon(pointListSideOuter)
    sideInner2D = geom.add_polygon(pointListSideInner)
    
    side2D = geom.boolean_difference(sideOuter2D, sideInner2D)
    
    side3D = geom.extrude(side2D, [0.0, 0.0, depth], num_layers=10)
    
    side_mesh = geom.generate_mesh()

In [8]:
# Create the back mesh
with pygmsh.occ.Geometry() as geom:

    geom.characteristic_length_max = 0.02

    back2D = geom.add_polygon(pointList)
    geom.translate(back2D, [0.0,0.0,depth])
    
    back3D = geom.extrude(back2D, [0.0, 0.0, topThickness], num_layers=2)
    
    back_mesh = geom.generate_mesh()

## Save Mesh to XDMF

In [23]:
# NEED TO FIX

# top_mesh_from_msh = meshio.read(os.path.join(MeshGeometryFolder, "top_mesh.msh"))
# tetra_top_mesh = create_mesh(top_mesh_from_msh, "tetra")
# meshio.write("top_mesh.xdmf", tetra_top_mesh)

# # Write the mesh
# meshio.write(os.path.join(MeshGeometryFolder,  "top_mesh.xml"), top_mesh)
# meshio.write(os.path.join(MeshGeometryFolder,  "side_mesh.xml"), side_mesh)
# meshio.write(os.path.join(MeshGeometryFolder,  "back_mesh.xml"), back_mesh)

# Gives Warnings!
# Warning: DOLFIN XML is a legacy format. Consider using XDMF instead.
# Warning: DOLFIN XML can only handle one cell type at a time. Using tetra, discarding vertex, triangle, line.

# top_mesh.write(os.path.join(MeshGeometryFolder,  "top_mesh.vtk"))
# side_mesh.write(os.path.join(MeshGeometryFolder, "side_mesh.vtk"))
# back_mesh.write(os.path.join(MeshGeometryFolder, "back_mesh.vtk"))

In [ ]:
# Documentation
# UFL: https://fenics.readthedocs.io/projects/ufl/en/latest/index.html
# DOLFINx: https://docs.fenicsproject.org/dolfinx/main/python/index.html